# Part I: DC Current

In [5]:
## Please run in Glowscript: http://www.glowscript.org/#/user/b06902080/folder/Private/program/final/edit

from vpython import *

## define constants
mu0_over4pi = 1e-7
pi = acos(-1.0)
R = 0.15
I = 50
qe = 1.602e-19
mass_e = 9.11e-31
c = 3e8

In [ ]:
enviro =canvas(width=600, height=400,title=  'click to shoot electron in')
apart = R
I1 = I2 = I

## Build the coils

I_ring1 = extrusion(path=paths.circle(pos=vec(0.5*apart,0,0),radius=R*1.1,up=vec(1,0,0)),
                   texture=textures.metal, color = color.yellow,
                    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
                            [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
                            [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])

con_rod = cylinder(pos=vector(0.5*R, -R, 0), axis = vector(0, -0.2*R, 0), radius = 0.04*R, color = color.gray(0.8), shininess = 0.6)
base = box(pos=vector(0.5*R, -1.3*R, 0), size=vector(0.6*R, 0.3*R, 0.6*R), color = vector(1, 0.7, 0.2))

coil1 = compound([con_rod, base])
coil2 = coil1.clone(pos = vec(-coil1.pos.x, coil1.pos.y, 0))
I_ring2 = I_ring1.clone(pos = -I_ring1.pos)


## discretize the coils
segment_Y = []
segment_Z = []
for i in range(0, 73):
    segment_Y.append( R*cos(i*2*pi/72) )
    segment_Z.append( R*sin(i*2*pi/72) )

## For x-axis field plotting
x_field = graph(width=800,height=400, xtitle = 'x_position (*R)', ytitle = 'Magnetic field (T)', ymax = 4e-4, ymin = -4e-4)
gc_B1 = gcurve(graph=x_field,color=color.green)
gc_B2 = gcurve(graph=x_field,color=color.blue)
gc_B = gcurve(graph=x_field,color=color.magenta)

def x_plot(dis):
    gc_B1.delete()
    gc_B2.delete()
    gc_B.delete()
    segment_pos = []
    dLength = []
    
    for i in range(0, 72):
        segment_pos.append( 0.5*vec(0.5*dis, segment_Y[i], segment_Z[i]) + 
                            0.5*vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) )
        dLength.append( vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) - 
                        vec(0.5*dis, segment_Y[i], segment_Z[i]) )

    ## compute B-field along x-axis
    for i in range(-200, 201):
        now_pos = vec(i*(dis/200),0,0)
        B_now = vec(0,0,0)
        B1 = B2 = vec(0, 0, 0)
        
        for j in range (0,72):
            pos1 = segment_pos[j]
            pos2 = vec(-pos1.x, pos1.y, pos1.z)
            dB1 = mu0_over4pi*I1*cross(dLength[j], now_pos - pos1) / mag(now_pos - pos1)**3
            dB2 = mu0_over4pi*I2*cross(dLength[j], now_pos - pos2) / mag(now_pos - pos2)**3
            B1 += dB1
            B2 += dB2
            B_now += dB1+dB2

        gc_B.plot([now_pos.x/R, B_now.x])
        gc_B1.plot([now_pos.x/R, B1.x])
        gc_B2.plot([now_pos.x/R, B2.x])

## adjustible distance between coils
x_plot(apart)
enviro.caption = "Vary the distance between coils\n"
def setapart(s):
    wt.text = '{:1.2f}'.format(s.value)
    global apart
    apart = s.value*R
    coil1.pos.x = 0.5*apart
    coil2.pos.x = -0.5*apart
    I_ring1.pos.x = 0.5*apart
    I_ring2.pos.x = -0.5*apart
    x_plot(apart)
    
sl_ap = slider(min=0.5, max=3, value=1, length=220, bind=setapart, right=15)
wt = wtext(text='{:1.2f}'.format(sl_ap.value))
enviro.append_to_caption('* Radius of coil\n')

## adjust I1 current
enviro.append_to_caption('Vary the current on coil1 (right)\n')
def seti1(s):
    wti1.text = '{:1d}'.format(s.value)
    global I1
    I1 = s.value
    x_plot(apart)

sl_i1 = slider(min=-100, max=100, value=50, length=220, bind=seti1, right=15)
wti1 = wtext(text='{:1d}'.format(sl_i1.value))
enviro.append_to_caption(' Ampere\n')

## adjust I2 current
enviro.append_to_caption('Vary the current on coil2 (right)\n')
def seti2(s):
    wti2.text = '{:1d}'.format(s.value)
    global I2
    I2 = s.value
    x_plot(apart)

sl_i2 = slider(min=-100, max=100, value=50, length=220, bind=seti2, right=15)
wti2 = wtext(text='{:1d}'.format(sl_i2.value))
enviro.append_to_caption(' Ampere\n')

def B_calculate(pos):
    B_now = vec(0,0,0)
    for i in range (0,72):
        pos1 = ( 0.5*vec(0.5*apart, segment_Y[i], segment_Z[i]) + 
                 0.5*vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) )
        pos2 = vec(-pos1.x, pos1.y, pos1.z)
        dLength = ( vec(0.5*apart, segment_Y[i+1], segment_Z[i+1]) - 
                    vec(0.5*apart, segment_Y[i], segment_Z[i]) )
        dB1 = mu0_over4pi*I1*cross(dLength, pos - pos1) / mag(pos - pos1)**3
        dB2 = mu0_over4pi*I2*cross(dLength, pos - pos2) / mag(pos - pos2)**3
        B_now += dB1+dB2
    return B_now

## send an electron in
electron = sphere(pos=vec(0, 0, 0), radius = 0.05*R, make_trail = True, retain=400)
enviro.waitfor('click')

t = 0
dt = 1e-11
e_velo = vec(0, -7e-3*c, 0)
while(mag(electron.pos) <= 5*R):
    rate(100000)
    Fmag = -qe*cross(e_velo, B_calculate(electron.pos))
    e_velo += Fmag/mass_e*dt
    electron.pos += e_velo*dt
    t += dt

<IPython.core.display.Javascript object>